# Módulo 3: OCR Inteligente con Amazon Textract

**Duración**: 15 minutos  
**Objetivo**: Digitalizar documentos científicos usando OCR avanzado

## ¿Qué aprenderás?
- Convertir documentos escaneados a texto digital
- Extraer tablas de datos automáticamente
- Procesar formularios de campo científicos
- Crear datasets estructurados desde PDFs

## Configuración
Textract requiere documentos en formato imagen o PDF.

In [ ]:
# Configuración específica para Textract
import boto3
import json
import pandas as pd
from PIL import Image, ImageDraw
import requests
import io

# Cliente Textract
textract = boto3.client('textract', region_name='us-east-1')
print("Textract configurado para OCR")

## Documentos Científicos para OCR

Trabajaremos con documentos que contienen datos estructurados típicos de investigación científica.

In [ ]:
# Documentos científicos simulados para OCR
SCIENTIFIC_DOCS = {
    "tabla_mediciones": {
        "url": "https://via.placeholder.com/800x600/ffffff/000000?text=MEDICIONES+SISMICAS%0A%0AEstacion+Fecha+Magnitud+Profundidad%0AEST01+2024-11-15+6.2+45km%0AEST02+2024-11-15+4.1+32km%0AEST03+2024-11-16+3.8+28km%0AEST04+2024-11-16+5.5+41km",
        "tipo": "tabla"
    },
    "formulario_campo": {
        "url": "https://via.placeholder.com/600x800/f8f9fa/212529?text=FORMULARIO+DE+CAMPO%0A%0AProyecto:+Monitoreo+Sismico%0AFecha:+15/11/2024%0AInvestigador:+Dr.+Martinez%0A%0AUbicacion:%0ALatitud:+-12.0464%0ALongitud:+-77.0428%0A%0AObservaciones:%0ASe+registro+actividad+sismica%0Aintensa+durante+la+madrugada%0A%0AFirma:+[Signature]",
        "tipo": "formulario"
    },
    "reporte_escanado": {
        "url": "https://via.placeholder.com/700x900/e9ecef/495057?text=INSTITUTO+GEOFISICO%0A%0AREPORTE+TECNICO+No.+2024-156%0A%0AANALISIS+SISMICO+REGIONAL%0A%0AResumen+Ejecutivo:%0AEl+presente+reporte+analiza+la%0Aactividad+sismica+registrada%0Aen+la+region+durante+el%0Aperido+del+10+al+15+de%0Anoviembre+de+2024.%0A%0AResultados:%0A-+Total+de+eventos:+47%0A-+Magnitud+maxima:+6.2+Mw%0A-+Profundidad+promedio:+38+km%0A%0AConclusiones:%0ALa+actividad+sismica+se%0Amantiene+dentro+de+parametros%0Anormales+para+la+region.",
        "tipo": "documento"
    }
}

print("Documentos científicos cargados para OCR:")
for nombre, info in SCIENTIFIC_DOCS.items():
    print(f"  • {nombre.replace('_', ' ').title()}: {info['tipo']}")

## Motor OCR de Textract

In [ ]:
def procesar_documento_ocr(doc_info, nombre_doc):
    """
    Procesa un documento usando el OCR de Textract
    """
    print(f"\n🔍 Procesando: {nombre_doc}")
    print(f"Tipo: {doc_info['tipo']}")
    
    try:
        # Descargar documento
        response = requests.get(doc_info['url'])
        doc_bytes = response.content
        
        # Análisis con Textract según el tipo
        if doc_info['tipo'] in ['tabla', 'formulario']:
            # Análisis completo con tablas y formularios
            resultado = textract.analyze_document(
                Document={'Bytes': doc_bytes},
                FeatureTypes=['TABLES', 'FORMS']
            )
        else:
            # Solo detección de texto
            resultado = textract.detect_document_text(
                Document={'Bytes': doc_bytes}
            )
        
        # Extraer información
        datos_extraidos = {
            'nombre': nombre_doc,
            'tipo': doc_info['tipo'],
            'texto_completo': '',
            'tablas': [],
            'campos_formulario': {},
            'bloques': resultado.get('Blocks', [])
        }
        
        # Procesar bloques de texto
        lineas_texto = []
        for bloque in datos_extraidos['bloques']:
            if bloque['BlockType'] == 'LINE':
                lineas_texto.append(bloque['Text'])
        
        datos_extraidos['texto_completo'] = '\n'.join(lineas_texto)
        
        # Extraer tablas si existen
        if doc_info['tipo'] == 'tabla':
            tablas = extraer_tablas_estructuradas(resultado)
            datos_extraidos['tablas'] = tablas
        
        print(f"✓ OCR completado: {len(datos_extraidos['texto_completo'])} caracteres")
        print(f"✓ Tablas encontradas: {len(datos_extraidos['tablas'])}")
        
        return datos_extraidos
        
    except Exception as e:
        print(f"✗ Error en OCR: {e}")
        return None

def extraer_tablas_estructuradas(respuesta_textract):
    """
    Extrae tablas de manera estructurada desde Textract
    """
    bloques = respuesta_textract['Blocks']
    tablas_extraidas = []
    
    # Mapear bloques por ID
    mapa_bloques = {bloque['Id']: bloque for bloque in bloques}
    
    # Encontrar bloques de tabla
    for bloque in bloques:
        if bloque['BlockType'] == 'TABLE':
            tabla_datos = procesar_tabla_individual(bloque, mapa_bloques)
            if tabla_datos:
                tablas_extraidas.append(tabla_datos)
    
    return tablas_extraidas

def procesar_tabla_individual(bloque_tabla, mapa_bloques):
    """
    Procesa una tabla individual de Textract
    """
    filas = {}
    
    if 'Relationships' in bloque_tabla:
        for relacion in bloque_tabla['Relationships']:
            if relacion['Type'] == 'CHILD':
                for id_hijo in relacion['Ids']:
                    bloque_hijo = mapa_bloques[id_hijo]
                    if bloque_hijo['BlockType'] == 'CELL':
                        fila = bloque_hijo.get('RowIndex', 1)
                        columna = bloque_hijo.get('ColumnIndex', 1)
                        texto_celda = obtener_texto_celda(bloque_hijo, mapa_bloques)
                        
                        if fila not in filas:
                            filas[fila] = {}
                        filas[fila][columna] = texto_celda
    
    # Convertir a lista ordenada
    tabla_ordenada = []
    for fila_num in sorted(filas.keys()):
        fila_datos = []
        for col_num in sorted(filas[fila_num].keys()):
            fila_datos.append(filas[fila_num][col_num])
        tabla_ordenada.append(fila_datos)
    
    return tabla_ordenada

def obtener_texto_celda(bloque_celda, mapa_bloques):
    """
    Obtiene el texto de una celda
    """
    texto = ''
    if 'Relationships' in bloque_celda:
        for relacion in bloque_celda['Relationships']:
            if relacion['Type'] == 'CHILD':
                for id_hijo in relacion['Ids']:
                    bloque_hijo = mapa_bloques[id_hijo]
                    if bloque_hijo['BlockType'] == 'WORD':
                        texto += bloque_hijo['Text'] + ' '
    return texto.strip()

print("Motor OCR de Textract configurado")

## Ejercicio 1: Digitalizar Tabla de Mediciones

In [ ]:
# Procesar tabla de mediciones sísmicas
tabla_resultado = procesar_documento_ocr(
    SCIENTIFIC_DOCS['tabla_mediciones'], 
    'Tabla de Mediciones Sísmicas'
)

if tabla_resultado and tabla_resultado['tablas']:
    print("\n📊 TABLA DIGITALIZADA:")
    print("=" * 50)
    
    # Convertir a DataFrame
    datos_tabla = tabla_resultado['tablas'][0]
    if len(datos_tabla) > 1:
        df_mediciones = pd.DataFrame(datos_tabla[1:], columns=datos_tabla[0])
        print(df_mediciones)
        
        # Análisis rápido
        print(f"\n📈 ANÁLISIS RÁPIDO:")
        print(f"• Total de registros: {len(df_mediciones)}")
        print(f"• Estaciones monitoreadas: {df_mediciones.iloc[:, 0].nunique()}")
        
        # Guardar como CSV
        df_mediciones.to_csv('mediciones_sismicas_ocr.csv', index=False)
        print(f"• Archivo guardado: mediciones_sismicas_ocr.csv")
else:
    print("No se pudieron extraer tablas del documento")

## Ejercicio 2: Procesar Formulario de Campo

In [ ]:
# Procesar formulario de campo científico
formulario_resultado = procesar_documento_ocr(
    SCIENTIFIC_DOCS['formulario_campo'], 
    'Formulario de Campo'
)

if formulario_resultado:
    print("\n📋 FORMULARIO DIGITALIZADO:")
    print("=" * 50)
    
    texto_formulario = formulario_resultado['texto_completo']
    lineas = texto_formulario.split('\n')
    
    # Extraer campos específicos
    campos_extraidos = {}
    for linea in lineas:
        if ':' in linea:
            campo, valor = linea.split(':', 1)
            campos_extraidos[campo.strip()] = valor.strip()
    
    # Mostrar campos estructurados
    for campo, valor in campos_extraidos.items():
        print(f"• {campo}: {valor}")
    
    # Crear registro estructurado
    registro_campo = pd.DataFrame([campos_extraidos])
    registro_campo.to_csv('registro_campo_ocr.csv', index=False)
    print(f"\n• Registro guardado: registro_campo_ocr.csv")
else:
    print("No se pudo procesar el formulario")

## Ejercicio 3: OCR de Reporte Técnico

In [ ]:
# Procesar reporte técnico escaneado
reporte_resultado = procesar_documento_ocr(
    SCIENTIFIC_DOCS['reporte_escanado'], 
    'Reporte Técnico Escaneado'
)

if reporte_resultado:
    print("\n📄 REPORTE DIGITALIZADO:")
    print("=" * 50)
    
    texto_reporte = reporte_resultado['texto_completo']
    
    # Mostrar texto extraído
    print(texto_reporte)
    
    # Estadísticas del documento
    palabras = len(texto_reporte.split())
    lineas = len(texto_reporte.split('\n'))
    caracteres = len(texto_reporte)
    
    print(f"\n📊 ESTADÍSTICAS DEL DOCUMENTO:")
    print(f"• Caracteres: {caracteres}")
    print(f"• Palabras: {palabras}")
    print(f"• Líneas: {lineas}")
    
    # Guardar texto completo
    with open('reporte_tecnico_ocr.txt', 'w', encoding='utf-8') as f:
        f.write(texto_reporte)
    print(f"• Texto guardado: reporte_tecnico_ocr.txt")
else:
    print("No se pudo procesar el reporte")

## Comparación: Antes vs Después del OCR

In [ ]:
# Resumen de digitalización
print("📈 RESUMEN DE DIGITALIZACIÓN")
print("=" * 60)

documentos_procesados = [
    ('tabla_mediciones', tabla_resultado),
    ('formulario_campo', formulario_resultado),
    ('reporte_escanado', reporte_resultado)
]

total_caracteres = 0
total_tablas = 0
archivos_generados = []

for nombre, resultado in documentos_procesados:
    if resultado:
        caracteres = len(resultado['texto_completo'])
        tablas = len(resultado['tablas'])
        
        total_caracteres += caracteres
        total_tablas += tablas
        
        print(f"\n• {nombre.replace('_', ' ').title()}:")
        print(f"  - Caracteres extraídos: {caracteres}")
        print(f"  - Tablas procesadas: {tablas}")
        print(f"  - Tipo: {resultado['tipo']}")

print(f"\n🎯 TOTALES:")
print(f"• Caracteres digitalizados: {total_caracteres}")
print(f"• Tablas estructuradas: {total_tablas}")
print(f"• Documentos procesados: {len([r for _, r in documentos_procesados if r])}")

print(f"\n📁 ARCHIVOS GENERADOS:")
archivos_ocr = [
    'mediciones_sismicas_ocr.csv',
    'registro_campo_ocr.csv', 
    'reporte_tecnico_ocr.txt'
]

for archivo in archivos_ocr:
    print(f"• {archivo}")

print("\n✓ Digitalización completada con Textract")

## Casos de Uso en Investigación Científica

### Textract para Ciencias:
- **Digitalización de archivos históricos**: Convertir papers antiguos a texto buscable
- **Extracción de datos tabulares**: Procesar tablas de mediciones y observaciones
- **Automatización de formularios**: Digitalizar registros de campo y laboratorio
- **Procesamiento de reportes escaneados**: Convertir documentos PDF a datos estructurados
- **Creación de datasets**: Generar CSV y bases de datos desde documentos físicos
- **Preservación digital**: Mantener información científica en formato digital

## Validación del Módulo

In [ ]:
def validar_modulo_textract():
    verificaciones = {
        "Textract configurado": 'textract' in globals(),
        "Tabla digitalizada": False,
        "Formulario procesado": False,
        "Reporte convertido": False
    }
    
    # Verificar resultados
    if 'tabla_resultado' in globals() and tabla_resultado:
        verificaciones["Tabla digitalizada"] = len(tabla_resultado.get('tablas', [])) > 0
    
    if 'formulario_resultado' in globals() and formulario_resultado:
        verificaciones["Formulario procesado"] = len(formulario_resultado.get('texto_completo', '')) > 0
    
    if 'reporte_resultado' in globals() and reporte_resultado:
        verificaciones["Reporte convertido"] = len(reporte_resultado.get('texto_completo', '')) > 0
    
    print("VALIDACIÓN MÓDULO 3 - TEXTRACT OCR")
    print("=" * 45)
    
    for verificacion, estado in verificaciones.items():
        icono = "✓" if estado else "✗"
        print(f"{icono} {verificacion}")
    
    completadas = sum(verificaciones.values())
    total = len(verificaciones)
    porcentaje = (completadas/total)*100
    
    print(f"\nProgreso: {completadas}/{total} ({porcentaje:.0f}%)")
    
    if completadas >= 3:
        print("\n🎉 ¡MÓDULO TEXTRACT COMPLETADO!")
        print("Has digitalizado documentos científicos exitosamente")
        print("➡️ Continúa con Módulo 4: Polly")
    else:
        print("\n⚠️ Completa más ejercicios de OCR")
    
    return completadas >= 3

modulo_completado = validar_modulo_textract()

---

## Próximo Módulo

**Módulo 4: Síntesis de Voz con Amazon Polly**

---

*Módulo 3 de 6 completado*